In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
import joblib

USER_ID = 353
PATH_DATA = '../data/user_sequences/'
EXTENSION_TEXT = '.txt'
PATH_USER_DATA = PATH_DATA + f'user_{USER_ID}_sequence' + EXTENSION_TEXT

data = pd.read_csv(PATH_USER_DATA, header=None)
data = data.to_numpy()[:,1:]  # Discard weekday, just for now
print(data[0:3,:])

[[38.7364443 -9.1407651]
 [38.7356124 -9.1396516]
 [38.7368552 -9.1396499]]


In [2]:
print(data.shape)
#X = data[0:-1,:]
#y = data[1:,:]

X = data
y = data

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = scaler.transform(y)

joblib.dump(scaler, filename='..\\models\\test\\scaler')

X_train = X[0:190,:]
y_train = y[0:190,:]

X_test = X[190:,:]
y_test = y[190:,:]

sequence_length = 3
train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=16)
test_generator = TimeseriesGenerator(X_test, y_test, length=3, batch_size=4)

(240, 2)


In [3]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(sequence_length, 2), return_sequences=True))
lstm_model.add(LSTM(50))
lstm_model.add(Dense(2))
lstm_model.compile(optimizer='adam', loss='mse')

epochs = 1000
lstm_history = lstm_model.fit(train_generator, epochs=epochs, verbose=1, batch_size=4)

Epoch 1/1000
12/12 [==============================] - 3s 5ms/step - loss: 1.0230
Epoch 2/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0163
Epoch 3/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.0064
Epoch 4/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.9950
Epoch 5/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.9788
Epoch 6/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.9578
Epoch 7/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.9305
Epoch 8/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8942
Epoch 9/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8596
Epoch 10/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8265
Epoch 11/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8043
Epoch 12/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.7929
Epoch 13/1000
12/12 [====

In [4]:
lstm_score = lstm_model.evaluate(test_generator, verbose=0)

#print(test_generator[1][0])
#print(test_generator[1][1])

#pred = lstm_model.predict(test_generator[1][0])
#print(pred)


pred_X = np.expand_dims(X_test[0:3, :], 0)
pred = lstm_model.predict(pred_X)
print(pred)
print(X_test[4,:])

print(lstm_score)
lstm_model.save('..\\models\\test\\model.h5')

1/1 [==============================] - 1s 523ms/step
[[1.3476741 1.6729971]]
[-0.92155375 -0.82353501]
1.426142930984497
